<a href="https://colab.research.google.com/github/venureddyjj/blank-app/blob/main/ATTACK_CAPEC_AUG17_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pwd

'/content/drive/My Drive'

In [13]:
cd /content/drive/MyDrive


/content/drive/MyDrive


In [14]:
import pandas as pd
import json

In [15]:
df_attack_main = pd.read_json('/content/drive/MyDrive/cti/enterprise-attack/enterprise-attack.json')

In [ ]:
df_attack_main

In [17]:
df_attack_db = pd.json_normalize(df_attack_main['objects'])

In [ ]:
df_attack_db

In [19]:
df_attack_attack_pattern=df_attack_db[df_attack_db['type'].str.contains('attack-pattern')]

In [20]:
len(df_attack_attack_pattern)

768

In [21]:
attack_depre_name = df_attack_attack_pattern['x_mitre_deprecated'].unique()

In [22]:
attack_revo_name = df_attack_attack_pattern['revoked'].unique()

In [23]:
values_to_remove = [True, ]

In [24]:
attack_pattern_final = df_attack_attack_pattern[~df_attack_attack_pattern['x_mitre_deprecated'].isin(values_to_remove)& ~df_attack_attack_pattern['revoked'].isin(values_to_remove)]

In [25]:
len(attack_pattern_final)

625

**CAPEC**

In [26]:
df_capec_main = pd.read_json('/content/drive/MyDrive/cti/capec/2.1/stix-capec.json')

In [27]:
df_capec_db = pd.json_normalize(df_capec_main['objects'])

In [28]:
df_capec_attack_pattern=df_capec_db[df_capec_db['type'].str.contains('attack-pattern')]

In [29]:
capec_values_to_remove = ['Deprecated']

In [30]:
capec_attack_pattern_final = df_capec_attack_pattern[~df_capec_attack_pattern['x_capec_status'].isin(capec_values_to_remove)]

In [31]:
len(capec_attack_pattern_final)

559

In [32]:
capec_attack_pattern_attack = capec_attack_pattern_final[capec_attack_pattern_final['external_references'].apply(lambda x: any(ref.get('source_name') == 'ATTACK' for ref in x))]

In [34]:
len(capec_attack_pattern_attack)

177

In [35]:
stixeye_attack_capec = pd.concat([capec_attack_pattern_attack, attack_pattern_final])

In [36]:
len(stixeye_attack_capec)

802

**Used Case1:Brute Force: Password Guessing(T) or Password Brute Forcing(C)**

In [37]:
search_word = 'T1110.001'
mask = stixeye_attack_capec.apply(lambda x: x.astype(str).str.contains(search_word, case=False)).any(axis=1)

In [38]:
count = mask.sum()

In [39]:
print(f"Number of occurrences of '{search_word}': {count}")

Number of occurrences of 'T1110.001': 3


In [42]:
search_rows = stixeye_attack_capec[mask]

In [43]:
from IPython.display import display, HTML

In [44]:
search_capec_execution_flow_attack = pd.DataFrame(search_rows['x_capec_execution_flow'])

In [45]:
from bs4 import BeautifulSoup

In [46]:
def extract_table(html_str):
    if pd.isna(html_str):  # Check if the value is NaN
        return ''
    soup = BeautifulSoup(html_str, 'html.parser')
    table = soup.find('table')
    return str(table)

In [48]:
search_capec_execution_flow_attack['x_capec_execution_flow_table'] = search_capec_execution_flow_attack['x_capec_execution_flow'].apply(extract_table)

In [50]:
print("Brute Force: Password Guessing(T) or Password Brute Forcing(C)")
display(HTML(search_capec_execution_flow_attack.to_html(escape=False)))

Brute Force: Password Guessing(T) or Password Brute Forcing(C)


In [70]:
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

def search_and_extract_table(stixeye_attack_capec, search_word):
    # Apply a mask to find rows containing the search word
    mask = stixeye_attack_capec.apply(lambda x: x.astype(str).str.contains(search_word, case=False)).any(axis=1)
    count = mask.sum()  # Count the number of matching rows

    # Extract the matching rows
    search_rows = stixeye_attack_capec[mask]
    search_capec_execution_flow_attack = pd.DataFrame(search_rows['x_capec_execution_flow'])

    # Function to extract table from HTML string
    def extract_table(html_str):
        if pd.isna(html_str):  # Check if the value is NaN
            return ''
        soup = BeautifulSoup(html_str, 'html.parser')
        table = soup.find('table')
        return str(table)

    # Apply the extract_table function to the 'x_capec_execution_flow' column
    search_capec_execution_flow_attack['x_capec_execution_flow_table'] = search_capec_execution_flow_attack['x_capec_execution_flow'].apply(extract_table)

    # Display the result as HTML
    display(HTML(search_capec_execution_flow_attack.to_html(escape=False)))

    return search_capec_execution_flow_attack, count

# Example usage
# stixeye_attack_capec = pd.read_csv('your_dataframe.csv')  # Load your DataFrame
# search_word = 'T1110.001'
# search_and_extract_table(stixeye_attack_capec, search_word)


In [71]:
search_capec_execution_flow_attack, count = search_and_extract_table(stixeye_attack_capec, "T1110.001")

In [72]:
search_capec_execution_flow_attack, count = search_and_extract_table(stixeye_attack_capec, "Brute Force")

In [76]:
print(stixeye_attack_capec.loc[2])

created                                                 2014-06-23T00:00:00.000Z
definition_type                                                              NaN
id                             attack-pattern--92cdcd3d-d734-4442-afc3-4599f2...
spec_version                                                                 2.1
type                                                              attack-pattern
                                                     ...                        
first_seen                                                                   NaN
last_seen                                                                    NaN
x_mitre_first_seen_citation                                                  NaN
x_mitre_last_seen_citation                                                   NaN
x_mitre_collection_layers                                                    NaN
Name: 2, Length: 75, dtype: object


In [82]:
search_capec_execution_flow_attack, count = search_and_extract_table(stixeye_attack_capec, "T1195.001")

,x_capec_execution_flow,x_capec_execution_flow_table
1145,NaN,
1154,NaN,
1452,NaN,
1573,"Execution Flow Explore Determine the relevant open-source code project to target: The adversary will make the selection based on various criteria:Experiment Develop a plan for malicious contribution: The adversary develops a plan to contribute malicious code, taking the following into consideration:Exploit Execute the plan for malicious contribution: Write the code to be contributed based on the plan and then submit the contribution. Multiple commits, possibly using multiple identities, will help obscure the attack. Monitor the contribution site to try to determine if the code has been uploaded to the target system.",None
2185,NaN,
2337,NaN,
2374,"Execution Flow Explore Identify target: The adversary must first identify a target repository that is commonly used and whose owner/maintainer has either changed/deleted their username or transferred ownership of the repository and then deleted their account. The target should typically be a popular and widely used package, as to increase the scope of the attack.Experiment Recreate initial repository path: The adversary re-registers the account that was renamed/deleted by the target repository's owner/maintainer and recreates the target repository with malicious code intended to exploit an application. These steps may need to happen in reverse (i.e., recreate repository and then rename an existing account to the target account) if protections are in place to prevent repository reuse.Exploit Exploit victims: The adversary's malicious code is incorporated into applications that directly reference the initial repository, which further allows the adversary to conduct additional attacks.",None
1014,NaN,


In [78]:
from IPython.display import display

In [80]:
display(stixeye_attack_capec.iloc[352].to_frame().T)

,created,definition_type,id,spec_version,type,definition.statement,identity_class,modified,name,object_marking_refs,...,x_mitre_impact_type,x_mitre_effective_permissions,x_mitre_network_requirements,x_mitre_data_source_ref,aliases,first_seen,last_seen,x_mitre_first_seen_citation,x_mitre_last_seen_citation,x_mitre_collection_layers
1146,2018-04-18T17:59:24.739Z,NaN,attack-pattern--3f18edba-28f4-4bb9-82c3-8aa60d...,NaN,attack-pattern,NaN,NaN,2023-03-30T21:01:42.446Z,Supply Chain Compromise,[marking-definition--fa42a846-8d90-4e51-bc29-7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
